In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2.10.1
Num GPUs Available:  1


Data Preprocessing 

In [2]:
training_data = "data/Training"
testing_data = "data/Testing"

class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# Data preprocessing
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    training_data,
    image_size=(256, 256),
    batch_size=64,
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    testing_data,
    image_size=(256, 256),
    batch_size=64,
)

# Data augmentation
datagen = ImageDataGenerator(
      rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_datagen = datagen
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    training_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    training_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    testing_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='categorical'
)

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.
Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


Show graph

In [ ]:

# Code for plotting
# def show(images, labels):
#     plt.figure(figsize=(10, 10))
#     for i in range(16):
#         plt.subplot(4, 4, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.grid(False)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.xlabel(class_names[labels[i]])
#     plt.show()

# show(train_images, train_labels)

Model

In [12]:
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4, activation='softmax'))
print(model.summary())

# Use categorical crossentropy loss
loss = losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer='adam', loss=loss, metrics=metrics)

batch_size = 64
epochs = 40

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 128, 128, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 32, 32, 64)      

Fit

In [13]:
# Convert labels to categorical
# train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=4)
# test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=4)

# model.fit(train_generator, train_labels, epochs=epochs, batch_size=batch_size, verbose=2)
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
)


Epoch 1/40
72/72 [==============================] - 55s 763ms/step - loss: 1.0783 - accuracy: 0.4929 - val_loss: 1.0804 - val_accuracy: 0.6074
Epoch 2/40
72/72 [==============================] - 54s 754ms/step - loss: 0.8157 - accuracy: 0.6443 - val_loss: 0.9816 - val_accuracy: 0.6897
Epoch 3/40
72/72 [==============================] - 54s 754ms/step - loss: 0.7432 - accuracy: 0.6734 - val_loss: 0.9546 - val_accuracy: 0.6266
Epoch 4/40
72/72 [==============================] - 56s 780ms/step - loss: 0.7156 - accuracy: 0.6963 - val_loss: 0.8995 - val_accuracy: 0.6985
Epoch 5/40
72/72 [==============================] - 55s 769ms/step - loss: 0.6459 - accuracy: 0.7379 - val_loss: 0.8277 - val_accuracy: 0.6994
Epoch 6/40
72/72 [==============================] - 56s 774ms/step - loss: 0.6223 - accuracy: 0.7493 - val_loss: 0.8654 - val_accuracy: 0.6897
Epoch 7/40
72/72 [==============================] - 56s 772ms/step - loss: 0.5713 - accuracy: 0.7674 - val_loss: 0.7692 - val_accuracy: 0.7266

Evaulation

In [14]:
# model.evaluate(test_generator, test_labels, batch_size=batch_size, verbose=2)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'Test loss: {test_loss:.2f}')

21/21 [==============================] - 10s 449ms/step - loss: 0.3791 - accuracy: 0.8566
Test accuracy: 0.86
Test loss: 0.38
